#### [ 교차검증 (Cross Validation) - cross_val_score()/cross_validate() ]
- 적은 데이터셋으로 안정적이고 신뢰성있는 모델 평가를 위한 방법
- 학습 데이터셋을 k개 분할 후 매번 다른 데이터로 검증 진행
- 교차검증 후 모델의 일반화 성능으로 여김
- sklearn.model_selection 서브모듈에 존재하는 함수들
    * cross_val_score()/ cross_val_predict() : cv만큼에 대한 성능결과, 예측결과 반환
    * cross_validate() : 다양한 

[1] 모듈 로딩 및 데이터 준비 <hr>

In [24]:
## [1-1] 모듈 로딩
## 기본 모듈
import numpy as np
import pandas as pd

## ML
from sklearn.model_selection import cross_val_score, cross_validate  ## 교차검증용
from sklearn.neighbors import KNeighborsClassifier                   ## 학습 알고리즘

In [25]:
## [1-2] 데이터 준비
data_file = '../Data/iris.csv'

irisDF = pd.read_csv(data_file)
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 데이터 전처리 및 학습 준비 : 시간 관계상 미진행 => 개인별로 진행<hr>

In [26]:
##[2-1] 품종컬럼 자료형 변환
pd.options.mode.copy_on_write = True

irisDF.variety = irisDF.variety.astype('category')
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   sepal.length  150 non-null    float64 
 1   sepal.width   150 non-null    float64 
 2   petal.length  150 non-null    float64 
 3   petal.width   150 non-null    float64 
 4   variety       150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [27]:
## [2-2] 피쳐와 타겟분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]

print(f'featureDF:{featureDF.shape}, targetSR :{targetSR.shape}')


featureDF:(150, 4), targetSR :(150,)


In [28]:
## [2-3] 수치형 컬럼 => 학습알고리즘에 따라 적용
##                  => KNN 알고리즘은 거리측정 기반으로 스케일링 필요함!! 

[3] 교차검증 <hr>

In [ ]:
### ------------------------------------------------------------
## [3-1] cross_validatae() 함수 : 전달된 데이터에 따라서 자동으로
##                               KFold, StratifiedFold 설정
## - 필수 매개변수
##   estimator          : 모델 인스턴스
##   cv                 : 기본 5 또는 KFold, StratifiedFold 인스턴스
##   return_train_score : 학습용 데이터셋 성능 반환여부 설정
##   return_estimator   : 학습 모델
### ------------------------------------------------------------
# 모델 인스턴스 생성
neighbors = 5
resultDF = pd.DataFrame(columns=['fit_time' , 'score_time' , 'test_score' , 'train_score', 'neighbors'])
for neighbors in range(1, 21) :
        
    kModel = KNeighborsClassifier(n_neighbors=neighbors)

    ## 함수 호출
    resultDict = cross_validate(kModel, 
                                featureDF, 
                                targetSR,
                                return_train_score = True,
                                cv = 3)
    # print(resultDict)
    ret = [ resultDict[k].mean().item() for k in resultDict.keys()]
    ret.append(neighbors)
    resultDF.loc[resultDF.shape[0]] = ret
    resultDF['diff'] = abs(ret[-3] - ret[-2])
    # break
    


{'fit_time': array([0.00099874, 0.00150704, 0.00099945]), 'score_time': array([0.00200009, 0.00200677, 0.00200033]), 'test_score': array([0.98, 0.94, 0.96]), 'train_score': array([1., 1., 1.])}


In [30]:
resultDF.sort_values(by='diff').head()

,fit_time,score_time,test_score,train_score,neighbors,diff
0,0.001666,0.001367,0.96,1.0,1.0,0.04


In [31]:
### -------------------------------------------------------
## [3-2] StratifiedKFold교차 검증
##      => .split(2D_피쳐, 1D_타겟) : 타겟 클래스/라벨 비율 계산 사용
### -------------------------------------------------------
# k-fold 인스턴스 생성
kfold = StratifiedKFold(random_state=7, shuffle=True)
resultDF = pd.DataFrame(columns=['neighbors','train','valid', 'diff'])
max_n = 31
for n in range(1, max_n + 1 ):
    ts_scores=[]
    vs_scores=[]
## k개 교차검증 진행
## -> 학습용 k-1/k 인덱스, 검증용 1/k 인덱스
    for train_index, valid_index in kfold.split(featureDF, targetSR) :
        ## 학습용, 검증용 인게스 추출
        # print(f'x_train : {train_index.shape}, x_test : {train_index.shape}')
        
        ## 학습진행
        x_train, y_train = featureDF.iloc[train_index], targetSR[train_index]   ##   학습용 피쳐 및 타겟
        x_valid, y_valid = featureDF.iloc[valid_index], targetSR[valid_index]   ## 테스트용 피쳐및 타겟
        print(f'[Train 타겟별 비율 ] : {round(y_train.value_counts()/y_train.shape[0], 1)}')
        print(f'[Valid 타겟별 비율 ] : {round(y_valid.value_counts()/y_valid.shape[0], 1)}')
        
        kModel = KNeighborsClassifier(n_neighbors=n)
        kModel.fit(x_train, y_train)
        
        ## 검증 진행
        v_score = kModel.score(x_valid, y_valid)
        t_score = kModel.score(x_train, y_train)
        
        ts_scores.append(t_score)
        vs_scores.append(v_score)
        
    ## k-fold 진행 후 성능평균
    ts_mean ,vs_mean = sum(ts_scores)/len(ts_scores), sum(vs_scores)/len(vs_scores)
    # print(f'[neighbors = {n}]인 경우 => ts_scores 평균 :{ts_mean} / vs_scores 평균 : {vs_mean}')
    
    resultDF.loc[resultDF.shape[0]] = [n, ts_mean, vs_mean, abs(ts_mean - vs_mean)]
    
    
    



[Train 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Valid 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Train 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Valid 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Train 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Valid 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Train 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Valid 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Train 타겟별 비율 ] : variety
Setosa        0.3
Versicolor    0.3
Virginica     0.3
Name: count, dtype: float64
[Valid 타겟별 비율 ] : variety
Se

In [32]:
resultDF.sort_values(by='diff').head()

,neighbors,train,valid,diff
6,7.0,0.975000,0.973333,0.001667
4,5.0,0.975000,0.973333,0.001667
11,12.0,0.975000,0.973333,0.001667
10,11.0,0.976667,0.973333,0.003333
12,13.0,0.970000,0.973333,0.003333


In [33]:
## 최종 최적의 hyper-parameter : 5 => 최근접 이웃 알고리즘 특성에 따라서 결정!!
final_k = 5


[4] 최종 모델 생성 <hr>

In [34]:

from sklearn.model_selection import train_test_split


In [35]:
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR)

## 모델 인스턴스 생성
final_model = KNeighborsClassifier(n_neighbors=final_k)
final_model2 = KNeighborsClassifier(n_neighbors=final_k)
## 학습 진행
final_model.fit(x_train, y_train)
final_model2.fit(featureDF, targetSR)


,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


[5] 최종 테스트용 데이터셋으로 평가 진행 <hr>

In [36]:
## 모델 평가
final_score = final_model.score(x_test, y_test)
final2_score = final_model2.score(featureDF, targetSR)

print(final_score)
print(final2_score)

0.9473684210526315
0.9666666666666667
